In [ ]:
import numpy as np
import random
import torch
from datasets import load_dataset, concatenate_datasets
from transformers import (
    BertTokenizerFast,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    AutoModelForTokenClassification,
)
from seqeval.metrics import accuracy_score, f1_score, precision_score, recall_score
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# 1. Load dataset and labels
try:
    dataset = load_dataset("conll2003")
    label_names = dataset["train"].features["ner_tags"].feature.names
    num_labels = len(label_names)
    logger.info("Dataset loaded successfully")
except Exception as e:
    logger.error(f"Error loading dataset: {e}")
    raise

# 2. Load tokenizer
try:
    tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
    logger.info("Tokenizer loaded successfully")
except Exception as e:
    logger.error(f"Error loading tokenizer: {e}")
    raise

# 3. Augmentation dictionary
ENTITY_DICT = {
    "PER": ["Barack Obama", "Angela Merkel", "Elon Musk", "Narendra Modi"],
    "LOC": ["Paris", "New York", "Delhi", "Tokyo"],
    "ORG": ["Google", "UN", "Tesla", "Apple"]
}

# 4. Entity swap augmentation
def swap_named_entities(example):
    tokens = example["tokens"]
    ner_tags = example["ner_tags"]
    new_tokens = []
    new_tags = []
    i = 0
    while i < len(tokens):
        tag = ner_tags[i]
        tag_name = label_names[tag]
        if tag_name.startswith("B-"):
            entity_type = tag_name[2:]
            entity_tokens = [tokens[i]]
            i += 1
            while i < len(tokens) and label_names[ner_tags[i]].startswith("I-"):
                entity_tokens.append(tokens[i])
                i += 1
            if entity_type in ENTITY_DICT:
                replacement = random.choice(ENTITY_DICT[entity_type]).split()
                new_tokens.extend(replacement)
                new_tags.extend([label_names.index("B-" + entity_type)] +
                                [label_names.index("I-" + entity_type)] * (len(replacement) - 1))
            else:
                new_tokens.extend(entity_tokens)
                new_tags.extend([label_names.index("B-" + entity_type)] +
                                [label_names.index("I-" + entity_type)] * (len(entity_tokens) - 1))
        else:
            new_tokens.append(tokens[i])
            new_tags.append(ner_tags[i])
            i += 1
    return {"tokens": new_tokens, "ner_tags": new_tags}

# 5. Augment + combine
try:
    augmented = dataset["train"].map(swap_named_entities)
    augmented_train_dataset = concatenate_datasets([dataset["train"], augmented])
    logger.info("Data augmentation completed")
except Exception as e:
    logger.error(f"Error during augmentation: {e}")
    raise

# 6. Tokenize and align
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, padding='max_length', is_split_into_words=True, max_length=128
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# 7. Tokenize datasets
try:
    tokenized_train = augmented_train_dataset.map(tokenize_and_align_labels, batched=True)
    tokenized_val = dataset["validation"].map(tokenize_and_align_labels, batched=True)
    logger.info("Tokenization completed")
except Exception as e:
    logger.error(f"Error during tokenization: {e}")
    raise

# 8. Load model
try:
    model = AutoModelForTokenClassification.from_pretrained(
        "bert-base-uncased", num_labels=num_labels
    )
    logger.info("Model loaded successfully")
except Exception as e:
    logger.error(f"Error loading model: {e}")
    raise

# 9. Evaluation metrics
def compute_metrics(p):
    preds, labels = p
    preds = np.argmax(preds, axis=2)
    true_preds = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]
    return {
        "accuracy": accuracy_score(true_labels, true_preds),
        "precision": precision_score(true_labels, true_preds),
        "recall": recall_score(true_labels, true_preds),
        "f1": f1_score(true_labels, true_preds),
    }

# 10. Training arguments
try:
    training_args = TrainingArguments(
        output_dir="./ner_model",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
        learning_rate=3e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),  # Enable mixed precision if GPU is available
    )
    logger.info("Training arguments set")
except Exception as e:
    logger.error(f"Error setting training arguments: {e}")
    raise

# 11. Trainer
try:
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_val,
        tokenizer=tokenizer,
        data_collator=DataCollatorForTokenClassification(tokenizer),
        compute_metrics=compute_metrics,
    )
    logger.info("Trainer initialized")
except Exception as e:
    logger.error(f"Error initializing trainer: {e}")
    raise

# 12. Train
try:
    trainer.train()
    logger.info("Training completed successfully")
except Exception as e:
    logger.error(f"Error during training: {e}")
    raise

c:\Users\laksh\anaconda3\envs\ner_bert\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:__main__:Dataset loaded successfully
INFO:__main__:Tokenizer loaded successfully
INFO:__main__:Data augmentation completed
INFO:__main__:Tokenization completed
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:Model loaded successfully
INFO:__main__:Training arguments set
C:\Users\laksh\AppData\Local\Temp\ipykernel_39432\1564437038.py:170: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  t

Epoch,Training Loss,Validation Loss


In [ ]:
# 12. Save model
trainer.save_model("./ner_model_final")

In [22]:
import accelerate, transformers, torch, datasets, seqeval, numpy
print(accelerate.__version__, transformers.__version__, torch.__version__)


1.7.0 4.52.4 2.7.1+cpu


In [28]:
import sys
print(sys.executable)
print(sys.version)

c:\Users\laksh\anaconda3\envs\ner_bert\python.exe
3.12.11 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 12:58:53) [MSC v.1929 64 bit (AMD64)]


In [29]:
from transformers import Trainer, TrainingArguments
try:
    training_args = TrainingArguments(output_dir="./test", eval_strategy="epoch")
    print("TrainingArguments created successfully")
except Exception as e:
    print(f"Error: {e}")

Error: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`
